In [ ]:
!pip install tiktoken transformers_stream_generator einops optimum  auto-gptq
!pip install huggingface_hub
!pip install tiktoken
!pip install verovio
from transformers import AutoModelForCausalLM, AutoTokenizer
from transformers.generation import GenerationConfig
import warnings
warnings.filterwarnings("ignore")
import textwrap
import torch

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 453.7/453.7 kB 19.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.5/23.5 MB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.5/322.5 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 22.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.2/13.2 MB 28.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 4.6 MB/s eta 0:00:00
  Creat

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

model_id = "google/flan-t5-large"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForSeq2SeqLM.from_pretrained(model_id, device_map="auto", trust_remote_code=True).eval()


tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.13G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [ ]:
email = '''
Dear WidgetCorp,
I am writing to express my deep disappointment with your service regarding my recent order (Order
#98765). I had ordered the "SuperWidget" as a birthday gift for my daughter, Emily, whose birthday
was on March 15th. Despite your website promising delivery within three days, the product arrived
only yesterday, missing her birthday entirely.
I attempted to contact your customer service multiple times for an explanation and to expedite the
delivery, but to no avail. My calls and emails went unanswered, leaving me in a difficult position with
no gift for Emily’s special day.
This experience has left a sour taste, and I am reconsidering future purchases from WidgetCorp. I
expect a prompt explanation and an assurance that such delays will not occur in the future.
Sincerely,
Jane Doe
'''

Summarization

In [ ]:
# Prepare the prompt for summarization
prompt = f"""
Please summarize the following email in 30 words or less:

Email:
{email}
"""

# Tokenize the input prompt
inputs = tokenizer(prompt, return_tensors="pt")

# Generate the summary with the model
outputs = model.generate(**inputs, max_length=50)  # Adjust max_length as needed

# Decode the generated summary
summary = tokenizer.decode(outputs[0], skip_special_tokens=True)

# Attempt to extract the summary more reliably
summary = summary.strip()  # Just take the whole summary

# Print the final summary
print("Final Summary:", summary)


Final Summary: Dear WidgetCorp, I am writing to express my deep disappointment with your service regarding my recent order (Order #98765). I had ordered the "SuperWidget" as a birthday gift for my daughter


Question/Answering

In [ ]:
prompt = f"""
Email:
```{email}```

Answer the following questions from the email delimited by triple backticks.
1. What was the reason for the customer's disappointment with WidgetCorp?
2. How did WidgetCorp's customer service respond to the customer's inquiries about the late
delivery?

"""
inputs = tokenizer(prompt, return_tensors="pt")
outputs = model.generate(**inputs, max_new_tokens=150)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

1. the product arrived only yesterday 2. My calls and emails went unanswered


Named-Entities

In [ ]:
prompt = f"""
Identify the entities in the email delimited by triple backticks (use standard NER terminology).

Email:
```{email}```
"""
inputs = tokenizer(prompt, return_tensors="pt")
outputs = model.generate(**inputs, max_new_tokens=150)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Jane Doe, WidgetCorp


English to French Translation

In [ ]:
prompt = f"""
Translate the following email delimited by triple backticks into French

Email:
```{email}```
"""
inputs = tokenizer(prompt, return_tensors="pt")
outputs = model.generate(**inputs, max_new_tokens=len(email)*2)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

 Monsieur WidgetCorp, je vous appelle à exprimer mon profond désapprochissement avec votre service en ce qui concerne mon achat récent (Ordonnance #98765). Je l'ai ordre du "SuperWidget" comme cadeau de naissance pour ma fille, Emily, dont l'anniversaire était le 15 mars. Bien que votre site vous a promis une livraison dans trois jours, le produit a arrivée à l'heure actuelle, et il n'a pas eu le dcor. Je l'ai tenté de communiquer avec votre service de clientèle plusieurs fois pour obtenir une explanation et obtenir une accélération de la livraison, mais à aucune fin. Même si j'ai eu plusieurs appels et e-mails, j'ai entraîné à l'inconvénient de ne pas avoir un cadeau pour Emily's anniversaire. Cette expérience a laissé une odeur soure et je révisera à l'avenir les achats de WidgetCorp. Je s'attends à une explanation et à une assurance que ces retards ne se trouvent pas à l'avenir.


French to English Translation

In [ ]:
prompt = f"""
Translate the following email delimited by triple backticks into English

Email traduit en français :
```
        Monsieur WidgetCorp, je vous appelle à exprimer mon profond désapprobation avec votre service en ce qui concerne mon achat récent (Ordonnance #98765). Je l'ai commandé du "SuperWidget" comme cadeau de naissance pour ma fille, Emily, dont l'anniversaire était le 15 mars. Bien que votre site vous ait promis une livraison dans trois jours, le produit est arrivé à ce jour, et il n'a pas été décoré. J'ai tenté de communiquer avec votre service clientèle plusieurs fois pour obtenir une explication et demander une accélération de la livraison, mais sans succès. Même si j'ai eu plusieurs appels et e-mails, j'ai été dans l'inconvénient de ne pas avoir de cadeau pour l'anniversaire d'Emily. Cette expérience a laissé un goût amer et je réviserai à l'avenir mes achats chez WidgetCorp. J'attends une explication et une assurance que ces retards ne se reproduiront pas à l'avenir.

Cordialement, Jane Doe
```
"""
inputs = tokenizer(prompt, return_tensors="pt")
outputs = model.generate(**inputs, max_new_tokens=len(email)*2)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

 Monsieur WidgetCorp, I call on you to express my deep disapproval with your service in regard to my recent purchase (Order #98765). I ordered the "SuperWidget" as a birthday gift for my daughter, Emily, whose birthday was on March 15th. While your site you had promised a delivery in three days, the product was delivered to this day, and it was not decorated. I tried to contact your customer service several times to obtain an explanation and request an acceleration of the delivery, but without success. Even though I have had several calls and emails, I was in the unfortunate situation of not having a birthday gift for Emily. This experience left a pleasant taste in my mouth and I re-visit my purchases at WidgetCorp. I await an explanation and assurance that these delays will not repeat in the future. Cordially, Jane Doe 
